#  프롬프트 엔지니어링 
- Chain-of-Thought (CoT) 등 고급 기법 

### **학습 목표:**  Chain-of-Thought 추론 기법을 활용한 복잡한 문제 해결 방법을 학습한다

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [1]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
    top_p=0.9,
)

In [3]:
from langchain_ollama import ChatOllama

ollama = ChatOllama(
    model='phi3:mini',
    temperature=0.3,
    top_p=0.9,
)

# **Chain of Thought (CoT)**

* Chain of Thought는 AI 모델이 복잡한 문제를 해결할 때 각 단계별 사고 과정을 명시적으로 보여주도록 하는 프롬프팅 기법으로, 이를 통해 모델의 추론 과정을 투명하게 확인할 수 있고 더 정확한 결과를 도출할 수 있습니다.

* 이 방식은 특히 수학 문제 풀이, 논리적 추론이 필요한 과제, 복잡한 의사결정 과정에서 매우 효과적이며, 모델이 중간 단계에서 발생할 수 있는 오류를 스스로 발견하고 수정할 수 있게 합니다.

* CoT의 주요 장점은 문제 해결 과정의 투명성을 높이고 최종 답변의 신뢰성을 향상시킬 수 있다는 것이지만, 출력이 길어지고 계산 비용이 증가할 수 있다는 단점도 존재합니다.


`(1) Zero-shot 프롬프팅`

   - 가장 단순한 형태의 프롬프팅
   - 예시나 단계별 설명 없이 직접 답을 출력
   - 속도가 빠르고 메모리 사용량이 적은 편
   - 단순한 문제에 적합

In [4]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
zero_shot_template = """
다음 문제를 해결하시오:

문제: {question}

답안:
"""

zero_shot_prompt = PromptTemplate(
    input_variables=["question"],
    template=zero_shot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(zero_shot_prompt.format(question=question))


다음 문제를 해결하시오:

문제: 
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?


답안:



In [5]:
# OpenAI gpt-4.1-mini 모델로 답안 생성

zero_shot_chain = zero_shot_prompt | llm 

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

문제에서 주어진 정보를 정리해보겠습니다.

- 전체 학생 수: 500명
- 5학년 학생 비율: 30% → 500 × 0.30 = 150명
- 6학년 학생 비율: 20% → 500 × 0.20 = 100명

5학년 학생 중:
- 수학 동아리: 60% → 150 × 0.60 = 90명
- 과학 동아리: 40% → 150 × 0.40 = 60명

6학년 학생 중:
- 수학 동아리: 70% → 100 × 0.70 = 70명
- 과학 동아리: 30% → 100 × 0.30 = 30명

과학 동아리 학생 수 = 5학년 과학 동아리 + 6학년 과학 동아리  
= 60명 + 30명 = 90명

---

**답안:**  
과학 동아리에는 총 90명의 학생이 있습니다.


In [6]:
# Ollama Phi3:mini 모델로 답안 생성

zero_shot_chain = zero_shot_prompt | ollama

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

600명의 5학년, 420명의 6학년 학생이 있습니다.

수학 동아리에는 5학년 학생 30%인 180명이 있으며, 6학년 학생 70%인 294명이 있습니다.

과학 동아리에는 5학년 학생 60%인 108명이 있으며, 6학년 학생 30%인 294명이 있습니다.

따라서, 과학 동아리에는 108명의 5학년 학생과 294명의 6학년 학생이 있으며, 合計 392명이 있습니다.


`(2) One-shot/Few-shot 프롬프팅`

   - 하나 이상의 예시를 통해 문제 해결 방식을 제시 
   - 유사한 예시를 통해 학습 효과를 기대
   - Zero-shot보다 더 정확한 결과를 얻을 수 있음
   - 중간 복잡도의 문제에 적합

   - 논문: https://arxiv.org/abs/2005.14165

In [7]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
one_shot_template = """
다음은 수학 문제를 해결하는 예시입니다:

예시 문제: 한 학급에 30명의 학생이 있습니다. 이 중 40%가 남학생이라면, 여학생은 몇 명인가요?

예시 풀이:
1) 먼저 남학생 수를 계산합니다:
   - 전체 학생의 40% = 30 x 0.4 = 12명이 남학생

2) 여학생 수를 계산합니다:
   - 전체 학생 수 - 남학생 수 = 30 - 12 = 18명이 여학생

따라서 여학생은 18명입니다.

이제 아래 문제를 같은 방식으로 해결하시오:

새로운 문제: {question}

답안:
"""

one_shot_prompt = PromptTemplate(
   input_variables=["question"],
   template=one_shot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(one_shot_prompt.format(question=question))


다음은 수학 문제를 해결하는 예시입니다:

예시 문제: 한 학급에 30명의 학생이 있습니다. 이 중 40%가 남학생이라면, 여학생은 몇 명인가요?

예시 풀이:
1) 먼저 남학생 수를 계산합니다:
   - 전체 학생의 40% = 30 x 0.4 = 12명이 남학생

2) 여학생 수를 계산합니다:
   - 전체 학생 수 - 남학생 수 = 30 - 12 = 18명이 여학생

따라서 여학생은 18명입니다.

이제 아래 문제를 같은 방식으로 해결하시오:

새로운 문제: 
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?


답안:



In [8]:
# OpenAI gpt-4.1-mini 모델로 답안 생성

one_shot_chain = one_shot_prompt | llm 

answer = one_shot_chain.invoke({"question": question})

print(answer.content)

답안:

1) 5학년 학생 수를 계산합니다:
   - 전체 학생의 30% = 500 x 0.3 = 150명

2) 6학년 학생 수를 계산합니다:
   - 전체 학생의 20% = 500 x 0.2 = 100명

3) 5학년 수학 동아리 학생 수를 계산합니다:
   - 5학년의 60% = 150 x 0.6 = 90명

4) 5학년 과학 동아리 학생 수를 계산합니다:
   - 5학년 전체 - 수학 동아리 = 150 - 90 = 60명

5) 6학년 수학 동아리 학생 수를 계산합니다:
   - 6학년의 70% = 100 x 0.7 = 70명

6) 6학년 과학 동아리 학생 수를 계산합니다:
   - 6학년 전체 - 수학 동아리 = 100 - 70 = 30명

7) 과학 동아리 학생 수를 모두 더합니다:
   - 5학년 과학 동아리 + 6학년 과학 동아리 = 60 + 30 = 90명

따라서 과학 동아리에는 90명의 학생이 있습니다.


In [9]:
# Ollama Phi3:mini 모델로 답안 생성

one_shot_chain = one_shot_prompt | ollama

answer = one_shot_chain.invoke({"question": question})

print(answer.content)

1) 5학년 학생의 수를 계산합니다:
   - 전체 학생의 30% = 500 x 0 end of text.
2) 6학년 학생의 수를 계산합니다:
   - 전체 학생의 20% = 500 x 0.2 = 100명이 6학년
3) 5학년 학생들의 수를 구합니다:
   - 전체 학생 수 - 6학년 학생 수 = 500 - 100 = 400명이 5학년
4) 5학년 학생들의 수를 구합니다:
   - 전체 학생 수 x 60% = 500 x 0.6 = 300명이 수학 동아리
   - 전체 학생 수 x 40% = 500 x 0.4 = 200명이 과학 동아리

따라서, 과학 동아리에는 200명의 학생이 있습니다.


`(3) Chain of Thought(CoT) 프롬프팅`

   - 가장 체계적인 문제 해결 방식을 제공
   - 명시적인 단계별 추론 과정을 포함
   - 복잡한 문제 해결에 가장 적합

   - 논문: https://arxiv.org/abs/2201.11903

In [10]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
cot_template = """
다음 문제를 논리적 단계에 따라 해결하시오:
문제: {question}

해결 과정:
1단계: 문제 이해하기
- 주어진 정보 파악
- 구해야 할 것 정리

2단계: 해결 방법 계획
- 사용할 수 있는 전략 검토
- 최적의 방법 선택

3단계: 계획 실행
- 선택한 방법 적용
- 중간 결과 확인

4단계: 검토
- 답안 확인
- 다른 방법 가능성 검토

답안:
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# 프롬프트 출력
print(cot_prompt.format(question=question))


다음 문제를 논리적 단계에 따라 해결하시오:
문제: 
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?


해결 과정:
1단계: 문제 이해하기
- 주어진 정보 파악
- 구해야 할 것 정리

2단계: 해결 방법 계획
- 사용할 수 있는 전략 검토
- 최적의 방법 선택

3단계: 계획 실행
- 선택한 방법 적용
- 중간 결과 확인

4단계: 검토
- 답안 확인
- 다른 방법 가능성 검토

답안:



In [11]:
# OpenAI gpt-4.1-mini 모델로 답안 생성

cot_chain = cot_prompt | llm 

answer = cot_chain.invoke({"question": question})

print(answer.content)

해결 과정:

1단계: 문제 이해하기  
- 주어진 정보:  
  - 전체 학생 수 = 500명  
  - 5학년 학생 비율 = 30% → 5학년 학생 수 = ?  
  - 6학년 학생 비율 = 20% → 6학년 학생 수 = ?  
  - 5학년 학생 중 60%는 수학 동아리, 나머지 40%는 과학 동아리  
  - 6학년 학생 중 70%는 수학 동아리, 나머지 30%는 과학 동아리  
- 구해야 할 것:  
  - 과학 동아리에 속한 학생 수

2단계: 해결 방법 계획  
- 5학년과 6학년 학생 수를 각각 구한다.  
- 각 학년별로 과학 동아리에 속한 학생 수를 계산한다.  
- 두 학년의 과학 동아리 학생 수를 합산한다.

3단계: 계획 실행  
- 5학년 학생 수 = 500 × 30% = 500 × 0.3 = 150명  
- 6학년 학생 수 = 500 × 20% = 500 × 0.2 = 100명  

- 5학년 과학 동아리 학생 수 = 150 × 40% = 150 × 0.4 = 60명  
- 6학년 과학 동아리 학생 수 = 100 × 30% = 100 × 0.3 = 30명  

- 과학 동아리 전체 학생 수 = 60 + 30 = 90명

4단계: 검토  
- 계산 과정이 논리적이고 정확한지 확인  
- 다른 학년 학생들은 동아리 정보가 없으므로 제외  
- 문제 조건에 맞는 답임을 확인

답안:  
과학 동아리에는 총 90명의 학생이 있습니다.


In [12]:
# Ollama Phi3:mini 모델로 답안 생성

cot_chain = cot_prompt | ollama

answer = cot_chain.invoke({"question": question})

print(answer.content)

1. 학교에서 학생의 수를 기준으로 5학년이 있는 학생의 수를 구합니다.
   - 5학년 학생 = 500 * 30% = 150명
2. 5학년 학생들에게서 수학 동아리에 있는 학생의 수를 구합니다.
   - 수학 동아리 60% = 150 * 60% = 90명
3. 6학년 학생들에게서 수학 동아리에 있는 학생의 수를 구합니다.
   end while
4. 6학년 학생들에서 과학 동아리에 있는 학생의 수를 구합니다.
   - 과학 동아리 30% = (500 * 20%) * 30% = 30명
5. 6학년 학생들에서 과학 동아리에 있는 학생의 수를 구합니다.
   - 과학 동아리 70% = (50nergies_in_science_club) * 70%
6. 두 개의 과학 동아리에서 각각 수학 동아리에 있는 학생의 수를 차례로 구합니다.
   - math_in_science_club = (5nergies_in0) * 70%
   - science_in_math_club = (5nergies_1) * 30%
7. 수학 동아리에서 과학 동아리에 있는 학생의 수를 구합니다.
8. 과학 동아리에서 두 개의 동아리에서 각각 수학 동아리에 있는 학생의 수를 차례로 구합니다.
   - math_in_science_club = (nergies_2) * 70%
   - science_in_math_club = (nergies_3) * 30%
9. 과학 동아리에서 두 개의 동아리에서 각각 수학 동아리에 있는 학생의 수를 구합니다.
   - math_in_science_club = (nergies_4) * 70%
   - science_in_math_club = (nergies_5) * 30%
10. 두 개의 과학 동아리에서 각각 수학 동아리에 있는 학생의 수를 구합니다.
   - math_in_science_club = (nergies_6) * 70%
11. 두 개의 과학 동아리에서 각각 수학 동아리에 있는 학생의 수를 구합니다.
   - math_in_science_club = (nerg

In [13]:
# 다른 오픈소스 모델로 테스트
from langchain_ollama import ChatOllama

gemma = ChatOllama(
    model='gemma2',
    temperature=0.3,
    top_p=0.9,
)

deepseek = ChatOllama(
    model='deepseek-r1:7b',
    temperature=0.3,
    top_p=0.9,
)

In [15]:
# GEMMA2 모델로 답안 생성

cot_chain = cot_prompt | gemma

answer = cot_chain.invoke({"question": question})

print(answer.content)

## 과학 동아리 학생 수 구하기 (논리적 단계별 해결)

**1단계: 문제 이해하기**

* **주어진 정보:**
    * 학교 학생 총 수: 500명
    * 5학년 학생 비율: 30%
    * 6학년 학생 비율: 20%
    * 5학년 중 수학 동아리 비율: 60%
    * 6학년 중 수학 동아리 비율: 70%
* **구해야 할 것:** 과학 동아리에 있는 학생 수

**2단계: 해결 방법 계획**

* **전략:**  각 학년별 과학 동아리 회원 수를 계산하고, 그 합을 구하는 방법.
    * 5학년과 6학년 각각의 수학 동아리 및 과학 동아리 구성원 수를 구한다.
    * 두 학년의 과학 동아리 구성원 수를 합하여 최종 결과를 얻는다.

**3단계: 계획 실행**

* **5학년:**
    * 5학년 학생 수: 500명 * 30% = 150명
    * 5학년 수학 동아리 학생 수: 150명 * 60% = 90명
    * 5학년 과학 동아리 학생 수: 150명 - 90명 = 60명
* **6학년:**
    * 6학년 학생 수: 500명 * 20% = 100명
    * 6학년 수학 동아리 학생 수: 100명 * 70% = 70명
    * 6학년 과학 동아리 학생 수: 100명 - 70명 = 30명

* **총 과학 동아리 학생 수:** 60명 + 30명 = 90명


**4단계: 검토**

* **답안 확인:** 문제에서 구하는 과학 동아리 학생 수는 90명입니다.
* **다른 방법 가능성 검토:**  문제 상황에 따라 다른 해결 방법이 있을 수 있습니다. 하지만 위의 논리적 단계를 따르면 명확하고 정확한 답을 얻을 수 있습니다.






In [18]:
# Ollama DeepSeek 모델로 답안 생성

zero_shot_chain = zero_shot_prompt | deepseek

answer = zero_shot_chain.invoke({"question": question})

print(answer.content)

<think>
먼저 학생의 총 수는 500명입니다. 30%는 5학년이므로그500 × 0.3 = 150명입니다. 
20%는 6학년이므ilog500 × 0.2 = 100명입니다. 
 science club에 속한 학생을 구하기 위해 5학년 학생 중 40%와 6학년 학생 중 30%의 학생 수를 구합니다. 
5학년 science club: 150 × 0.4 = 60명
6학년 science club: 100 × 0.3 = 30명
 science club에 속한 총 학생 수는 60 + 30 = 90명입니다.
</think>

**문제:**  
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다.  
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지 40%는 과학 동아리에 있습니다.  
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지 30%는 과학 동아리에 있습니다.  
과학동아리에는 몇 명의 학생이 있나요?

**풀이:**

1. **5학년 학생 수:**  
   \[
   500 \times 0.3 = 150 \text{명}
   \]

2. **6학년 학생 수:**  
   \[
   500 \times 0.2 = 100 \text{명}
   \]

3. **5학년 학생 중 과학동아리에 있는 학생 수:**  
   \[
   150 \times 0.4 = 60 \text{명}
   \]

4. **6학년 학생 중 과학동아리에 있는 학생 수:**  
   \[
   100 \times 0.3 = 30 \text{명}
   \]

5. **과학동아리에 있는 총 학생 수:**  
   \[
   60 + 30 = 90 \text{명}
   \]

**笞안:**  
\boxed{90}


### **[실습]** 다음 논리적 추론 문제를 phi-3:mini 모델을 사용하여 3가지 유형의 프롬프트를 작성하여 해결하고, 그 성능을 비교합니다. 

(정답: 7번 이동)

In [19]:
question = """
농부가 늑대, 양, 양배추를 데리고 강을 건너야 함

제약조건:
1. 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음
2. 농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림
3. 보트에는 농부와 한 물건만 실을 수 있음

모두 안전하게 건너는데 몇 번 이동이 필요할까?
"""

In [21]:
# [실습] phi-3:mini 모델로 zero-shot 방식 구현
from langchain_core.prompts import PromptTemplate

zero_shot_template = """
다음 문제를 해결하시오:

문제: {question}

답안:
"""

zero_shot_prompt = PromptTemplate(
    input_variables=["question"],
    template=zero_shot_template
)

zero_shot_chain = zero_shot_prompt | llm  # ollama는 phi3:mini로 설정됨
answer = zero_shot_chain.invoke({"question": question})
print("Zero-shot 답안:", answer.content)

Zero-shot 답안: 이 문제는 고전적인 '늑대, 양, 양배추 강 건너기' 문제입니다.

---

### 문제 요약
- 농부가 늑대, 양, 양배추를 강 건너편으로 모두 옮겨야 한다.
- 제약조건
  1. 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음
  2. 농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹음
  3. 보트에는 농부와 한 물건만 탈 수 있음 (즉, 농부는 항상 타야 하고, 농부 혼자 혹은 농부+한 물건만 이동 가능)

---

### 풀이 과정

농부가 강 건너편으로 모두 안전하게 옮기기 위한 최소 이동 횟수를 찾는 문제입니다.

- 농부가 항상 보트에 타야 하므로, 한 번 이동은 농부가 한쪽에서 다른 쪽으로 이동하는 것을 의미합니다.
- 농부가 혼자 이동하거나, 농부와 한 물건(늑대, 양, 양배추 중 하나)과 함께 이동할 수 있습니다.
- 중간에 늑대와 양만 남거나, 양과 양배추만 남으면 안 됩니다.

---

### 상태 표현

강 왼쪽과 오른쪽에 각각 누가 있는지 표시해보겠습니다.

- 초기 상태 (왼쪽): 농부, 늑대, 양, 양배추 모두 왼쪽에 있음
- 목표 상태 (오른쪽): 모두 오른쪽에 있음

---

### 이동 순서 (최소 이동)

1. 농부가 양을 데리고 오른쪽으로 이동  
   (왼쪽: 늑대, 양배추 / 오른쪽: 농부, 양)  
   - 왼쪽에 늑대와 양배추만 있으므로 안전

2. 농부 혼자 왼쪽으로 돌아감  
   (왼쪽: 농부, 늑대, 양배추 / 오른쪽: 양)  
   - 오른쪽에 양만 있으므로 안전

3. 농부가 늑대를 데리고 오른쪽으로 이동  
   (왼쪽: 양배추 / 오른쪽: 농부, 늑대, 양)  
   - 왼쪽에 양배추만 있으므로 안전

4. 농부가 양을 데리고 왼쪽으로 이동  
   (왼쪽: 농부, 양, 양배추 / 오른쪽: 늑대)  
   - 오른쪽에 늑대만 있으므로 안전

5. 농부이 양배추를 데리고 오른쪽으로 이동  
   (왼쪽: 양 / 오른쪽: 농부, 늑대, 양배추)  
   

In [22]:

# [실습] phi-3:mini 모델로 one-shot 방식 구현
one_shot_template = """
다음은 유사한 문제를 해결하는 예시입니다:

예시 문제: 농부가 양과 늑대를 데리고 강을 건너야 함. 제약: 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음. 보트에는 농부와 한 물건만 실을 수 있음. 몇 번 이동이 필요할까?

예시 풀이: 농부가 양을 데리고 건너고, 돌아와 늑대를 데리고 건너고, 양을 데리고 돌아와 양배추를 데리고 건너고, 돌아와 양을 데리고 건너야 함. 총 7번 이동.

이제 아래 문제를 해결하시오:

문제: {question}

답안:
"""

one_shot_prompt = PromptTemplate(
    input_variables=["question"],
    template=one_shot_template
)

one_shot_chain = one_shot_prompt | llm
answer = one_shot_chain.invoke({"question": question})
print("One-shot 답안:", answer.content)


One-shot 답안: 이 문제는 고전적인 강 건너기 퍼즐로, 농부가 늑대, 양, 양배추를 모두 안전하게 강 건너편으로 옮겨야 합니다. 제약조건 때문에 농부가 없을 때 늑대와 양이 같이 있으면 안 되고, 양과 양배추가 같이 있으면 안 됩니다. 보트에는 농부와 한 물건만 탈 수 있습니다.

---

### 문제 요약
- 등장인물: 농부, 늑대, 양, 양배추
- 제약조건:
  1. 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음
  2. 농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림
  3. 보트에는 농부와 한 물건만 탈 수 있음

---

### 풀이 과정

1. **농부가 양을 데리고 강 건너편으로 이동**  
   (왼쪽: 늑대, 양배추 / 오른쪽: 농부, 양)  
   - 양과 늑대만 남았으므로 안전  
   - 양과 양배추가 같이 있지 않으므로 안전

2. **농부 혼자 왼쪽으로 돌아옴**  
   (왼쪽: 농부, 늑대, 양배추 / 오른쪽: 양)  
   - 오른쪽에는 양만 있으므로 안전

3. **농부가 늑대를 데리고 강 건너편으로 이동**  
   (왼쪽: 양배추 / 오른쪽: 농부, 늑대, 양)  
   - 왼쪽에는 양배추만 있으므로 안전  
   - 오른쪽에는 농부가 있으므로 늑대와 양이 같이 있어도 안전

4. **농부가 양을 데리고 왼쪽으로 돌아옴**  
   (왼쪽: 농부, 양, 양배추 / 오른쪽: 늑대)  
   - 오른쪽에는 늑대만 있으므로 안전  
   - 왼쪽에는 농부가 있으므로 양과 양배추가 같이 있어도 안전

5. **농부가 양배추를 데리고 강 건너편으로 이동**  
   (왼쪽: 양 / 오른쪽: 농부, 늑대, 양배추)  
   - 왼쪽에는 양만 있으므로 안전  
   - 오른쪽에는 농부가 있으므로 늑대와 양배추가 같이 있어도 안전

6. **농부 혼자 왼쪽으로 돌아옴**  
   (왼쪽: 농부, 양 / 오른쪽: 늑대, 양배추)  
   - 오른쪽에는 늑대와 양배추가 같이 있으나 농부가 없으므로 문제

In [23]:

# [실습] phi-3:mini 모델로 CoT 방식 구현
cot_template = """
다음 문제를 단계별로 해결하시오:

문제: {question}

해결 과정:
1단계: 문제 이해 - 농부, 늑대, 양, 양배추의 안전한 이동.
2단계: 계획 - 농부가 물건을 옮기며 제약을 피함.
3단계: 실행 - 이동 순서를 나열.
4단계: 검토 - 총 이동 수 확인.

답안:
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template
)

cot_chain = cot_prompt | llm
answer = cot_chain.invoke({"question": question})
print("CoT 답안:", answer.content)


CoT 답안: 문제: 농부가 늑대, 양, 양배추를 데리고 강을 건너야 함  
제약조건:  
- 농부가 없을 때 늑대와 양이 같이 있으면 늑대가 양을 잡아먹음  
- 농부가 없을 때 양과 양배추가 같이 있으면 양이 양배추를 먹어버림  
- 보트에는 농부와 한 물건만 실을 수 있음  

---

## 1단계: 문제 이해  
농부, 늑대, 양, 양배추를 모두 안전하게 강 반대편으로 옮겨야 한다.  
농부가 없으면 늑대와 양이 같이 있으면 안 되고, 양과 양배추가 같이 있으면 안 된다.  
보트에는 농부와 한 물건만 탈 수 있다.  

---

## 2단계: 계획  
- 농부는 반드시 보트에 있어야 한다.  
- 농부가 양을 데리고 가거나, 늑대 또는 양배추를 데리고 가면서 위험 상황이 발생하지 않도록 순서를 정한다.  
- 위험 상황이 발생하지 않도록 중간에 농부가 혼자 돌아오는 경우도 고려한다.  

---

## 3단계: 실행 (이동 순서)  

초기 상태: 왼쪽 강가에 (농부, 늑대, 양, 양배추) 모두 있음  
목표 상태: 오른쪽 강가에 모두 있음  

| 이동 번호 | 보트 탑승자       | 왼쪽 강가 상태         | 오른쪽 강가 상태       | 비고                      |
|-----------|-------------------|------------------------|------------------------|---------------------------|
| 1         | 농부 + 양         | 늑대, 양배추           | 농부, 양               | 양을 먼저 옮김             |
| 2         | 농부 혼자         | 농부, 늑대, 양배추     | 양                     | 농부 혼자 돌아옴           |
| 3         | 농부 + 늑대       | 양배추                 | 농부, 늑대, 양       

# **Self-Consistency**

* Self-Consistency는 AI 모델에게 하나의 문제에 대해 다양한 접근 방식으로 해결하도록 요청하는 기법으로, 여러 경로를 통해 도출된 결과들의 일관성을 확인함으로써 답변의 신뢰성을 높입니다.

* 이 방법은 특히 수학 문제나 논리적 추론이 필요한 과제에서 효과적이며, 서로 다른 방법으로 도출된 결과가 일치하는지 검증함으로써 오류 가능성을 최소화할 수 있습니다.

* Self-Consistency의 장점은 답변의 정확성을 높일 수 있다는 것이지만, 여러 번의 계산과 추론이 필요하므로 처리 시간이 길어지고 컴퓨팅 리소스 사용량이 증가한다는 단점이 있습니다.

* 또한 이 기법은 Chain of Thought (CoT) 프롬프팅과 결합하여 사용할 경우 더욱 강력한 효과를 발휘할 수 있습니다.

- 논문: https://arxiv.org/abs/2203.11171


In [24]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
self_consistency_template = """
다음 문제를 세 가지 다른 방법으로 해결하시오:

문제: {question}

세 가지 풀이 방법:
1) 직접 계산 방법:
   - 주어진 숫자를 직접 계산

2) 비율 활용 방법:
   - 전체에 대한 비율로 계산

3) 단계별 분해 방법:
   - 문제를 작은 부분으로 나누어 계산

각 방법의 답안을 제시하고, 결과가 일치하는지 확인하시오.

답안:
"""

self_consistency_prompt = PromptTemplate(
   input_variables=["question"],
   template=self_consistency_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-mini 모델로 답안 생성
self_consistency_chain = self_consistency_prompt | llm 
answer = self_consistency_chain.invoke({"question": question})

print(answer.content)

문제:  
학교에 학생 500명  
- 5학년: 30% → 500 × 0.30 = 150명  
- 6학년: 20% → 500 × 0.20 = 100명  
- 5학년 중 수학 동아리: 60% → 150 × 0.60 = 90명  
- 5학년 중 과학 동아리: 40% → 150 × 0.40 = 60명  
- 6학년 중 수학 동아리: 70% → 100 × 0.70 = 70명  
- 6학년 중 과학 동아리: 30% → 100 × 0.30 = 30명  

과학 동아리 학생 수 = 5학년 과학 동아리 + 6학년 과학 동아리 = 60 + 30 = 90명

---

### 1) 직접 계산 방법

- 5학년 학생 수: 500 × 0.30 = 150명  
- 6학년 학생 수: 500 × 0.20 = 100명  
- 5학년 과학 동아리 학생 수: 150 × (1 - 0.60) = 150 × 0.40 = 60명  
- 6학년 과학 동아리 학생 수: 100 × (1 - 0.70) = 100 × 0.30 = 30명  
- 과학 동아리 총 학생 수: 60 + 30 = **90명**

---

### 2) 비율 활용 방법

- 5학년 과학 동아리 비율 = 30% × 40% = 0.30 × 0.40 = 0.12 (12%)  
- 6학년 과학 동아리 비율 = 20% × 30% = 0.20 × 0.30 = 0.06 (6%)  
- 전체 과학 동아리 비율 = 12% + 6% = 18%  
- 과학 동아리 학생 수 = 500 × 0.18 = **90명**

---

### 3) 단계별 분해 방법

1. 전체 학생 500명  
2. 5학년 학생 150명 (500 × 0.30)  
3. 5학년 수학 동아리 90명 (150 × 0.60)  
4. 5학년 과학 동아리 60명 (150 - 90)  
5. 6학년 학생 100명 (500 × 0.20)  
6. 6학년 수학 동아리 70명 (100 × 0.70)  
7. 6학년 과학 동아리 30명 (100 - 70)  
8. 과학 동아리 학생 수 = 60 + 30

# **Program-Aided Language (PAL)**

* PAL은 자연어 문제를 프로그래밍적 사고방식으로 접근하도록 하는 기법으로, 복잡한 문제를 코드나 의사코드 형태로 분해하여 해결하는 방식입니다. 이를 통해 문제 해결 과정을 더욱 구조화하고 체계적으로 만들 수 있습니다.

* 이 접근 방식의 큰 장점은 프로그래밍 언어의 정확성과 논리성을 활용하여 모호함을 줄이고, 각 단계를 명확하게 정의할 수 있다는 것입니다. 특히 수학적 계산, 데이터 처리, 알고리즘적 문제 해결에서 뛰어난 성능을 보입니다.

* PAL의 특징적인 점은 실제 실행 가능한 코드를 생성할 수 있다는 것으로, 이는 결과의 검증이 용이하고 필요한 경우 수정이나 최적화가 가능하다는 장점이 있습니다. 

- 논문: https://arxiv.org/pdf/2211.10435


In [25]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
pal_template = """
다음 문제를 Python 프로그래밍 방식으로 해결하시오:

문제: {question}

# 문제 해결을 위한 Python 스타일 의사코드:
def solve_problem():
    # 1. 변수 정의
    # - 주어진 값들을 변수로 저장
    
    # 2. 계산 과정
    # - 필요한 계산을 단계별로 수행
    
    # 3. 결과 반환
    # - 최종 결과 계산 및 반환
    
답안:
"""

pal_prompt = PromptTemplate(
    input_variables=["question"],
    template=pal_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-mini 모델로 답안 생성
pal_chain = pal_prompt | llm 
answer = pal_chain.invoke({"question": question})

print(answer.content)

```python
def solve_problem():
    # 1. 변수 정의
    total_students = 500
    percent_5th = 0.30
    percent_6th = 0.20

    percent_math_5th = 0.60
    percent_science_5th = 1 - percent_math_5th

    percent_math_6th = 0.70
    percent_science_6th = 1 - percent_math_6th

    # 2. 계산 과정
    num_5th = total_students * percent_5th
    num_6th = total_students * percent_6th

    science_5th = num_5th * percent_science_5th
    science_6th = num_6th * percent_science_6th

    total_science = science_5th + science_6th

    # 3. 결과 반환
    return int(total_science)

# 결과 출력
print(solve_problem())
```

**설명:**  
- 전체 학생 수에서 5학년과 6학년 학생 수를 구합니다.  
- 각 학년별로 수학 동아리와 과학 동아리 학생 수를 계산합니다.  
- 과학 동아리 학생 수를 합산하여 최종 결과를 반환합니다.


# **Reflexion**

* Reflexion은 AI가 자신의 이전 답변을 스스로 검토하고 평가하여 개선하는 메타인지적 프롬프팅 기법으로, 이를 통해 응답의 질을 점진적으로 향상시킬 수 있습니다.

* 이 방법은 AI가 자신의 답변에서 부족한 점, 오류, 또는 개선이 필요한 부분을 스스로 찾아내고 수정하도록 함으로써, 더 정확하고 완성도 높은 답변을 도출할 수 있게 합니다. 특히 복잡한 분석이나 창의적인 작업에서 효과적입니다.

* Reflexion의 강점은 AI가 자기 평가를 통해 지속적으로 개선된 결과물을 제공할 수 있다는 것이지만, 여러 번의 반복적인 검토와 수정 과정이 필요하므로 시간과 컴퓨팅 자원이 더 많이 소요될 수 있다는 제한점이 있습니다.

* 이 기법은 특히 글쓰기, 코드 리뷰, 분석 리포트 작성 등 높은 품질의 출력이 요구되는 작업에서 매우 유용하게 활용될 수 있습니다.

- 논문: https://arxiv.org/abs/2303.11366

In [26]:
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 생성
reflexion_template = """
다음 문제에 대해 단계적으로 해결하여 초기 답안을 작성하고, 자체 평가 후 개선하시오:

문제: {question}

1단계: 초기 답안
---
[여기에 첫 번째 답안 작성]

2단계: 자체 평가
---
- 정확성 검토
- 논리적 오류 확인
- 설명의 명확성 평가
- 개선이 필요한 부분 식별

3단계: 개선된 답안
---
[평가를 바탕으로 개선된 답안 작성]

답안:
"""

reflexion_prompt = PromptTemplate(
    input_variables=["question"],
    template=reflexion_template
)

# 테스트용 문제
question = """
학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
과학 동아리에는 몇 명의 학생이 있나요?
"""

# OpenAI gpt-4.1-mini 모델로 답안 생성
reflexion_chain = reflexion_prompt | ChatOpenAI(model='gpt-4.1-mini') 
answer = reflexion_chain.invoke({"question": question})

print(answer.content)

1단계: 초기 답안
---
주어진 조건에 따라 계산해보겠습니다.

1. 전체 학생 수는 500명입니다.
2. 5학년 학생 수 = 500명의 30% = 500 × 0.3 = 150명
3. 6학년 학생 수 = 500명의 20% = 500 × 0.2 = 100명

4. 5학년 중 수학 동아리 학생 수 = 150명의 60% = 150 × 0.6 = 90명  
   따라서 5학년 중 과학 동아리 학생 수 = 150 - 90 = 60명

5. 6학년 중 수학 동아리 학생 수 = 100명의 70% = 100 × 0.7 = 70명  
   따라서 6학년 중 과학 동아리 학생 수 = 100 - 70 = 30명

6. 과학 동아리 학생 수 = 5학년 과학 동아리 학생 + 6학년 과학 동아리 학생 = 60 + 30 = 90명

따라서 과학 동아리에는 총 90명의 학생이 있습니다.

2단계: 자체 평가
---
- 정확성 검토: 각 단계별 계산은 정확해 보인다. 퍼센트 계산과 합산이 올바르게 이루어졌다.
- 논리적 오류 확인: 명확하게 각 학년별 동아리 인원을 구하고, 과학 동아리 인원을 합산하여 논리 오류 없음.
- 설명의 명확성 평가: 모든 계산과정이 단계별로 차근차근 제시되어 이해하기 쉽다.
- 개선이 필요한 부분 식별: 문제 풀이 과정에 전체 학생이 5학년과 6학년 외에 다른 학년도 있을 수 있음을 언급하거나, 동아리에 속하지 않는 다른 학생이 있는지 여부를 확인할 필요가 있다. 문제에는 5학년과 6학년만 언급되어 있고, 동아리 참여 여부는 5학년과 6학년에 한정되어 있기 때문에 다른 학년 학생들의 동아리 가입 여부가 명확하지 않다. 그러나 문제에서 "과학 동아리에 몇 명이나 있나요?"라는 질문은 주어진 데이터로 5학년과 6학년만을 대상으로 계산하는 것이 합당해 보인다. 이 점을 간단히 언급하여 문제의 범위를 명확히 하는 것이 좋겠다.

3단계: 개선된 답안
---
학교에는 총 500명의 학생 중 5학년과 6학년 학생 일부가 있습니다.

1. 5학년 학생 수는 전체의 30%이

### **[실습 2]** [실습 1]의 논리적 추론 문제를 gemma2:2b 모델을 사용하여 3가지 유형의 프롬프트를 작성하여 해결하고, 그 성능을 비교합니다. 

(정답: 7번 이동)

In [27]:

# [실습 2] gemma2:2b 모델로 Self-Consistency 구현
self_consistency_template = """
다음 문제를 세 가지 방법으로 해결하시오:

문제: {question}

1) 직접 계산: 이동 순서를 나열.
2) 비율 활용: 각 단계의 비율로 계산.
3) 단계별 분해: 작은 단계로 나누어 계산.

각 방법의 답안을 제시하고 일치 여부 확인.

답안:
"""

self_consistency_prompt = PromptTemplate(
    input_variables=["question"],
    template=self_consistency_template
)

self_consistency_chain = self_consistency_prompt | llm  # gemma는 gemma2로 설정됨
answer = self_consistency_chain.invoke({"question": question})
print("Self-Consistency 답안:", answer.content)


Self-Consistency 답안: 문제:  
- 전체 학생 수 = 500명  
- 5학년 학생 비율 = 30% → 5학년 학생 수 = 500 × 0.30 = 150명  
- 6학년 학생 비율 = 20% → 6학년 학생 수 = 500 × 0.20 = 100명  
- 5학년 중 수학 동아리 비율 = 60% → 과학 동아리 비율 = 40%  
- 6학년 중 수학 동아리 비율 = 70% → 과학 동아리 비율 = 30%  

과학 동아리 학생 수를 구하시오.

---

### 1) 직접 계산: 이동 순서 나열

- 5학년 학생 수 = 500 × 0.30 = 150명  
- 5학년 과학 동아리 학생 수 = 150 × 0.40 = 60명  
- 6학년 학생 수 = 500 × 0.20 = 100명  
- 6학년 과학 동아리 학생 수 = 100 × 0.30 = 30명  
- 과학 동아리 전체 학생 수 = 60 + 30 = 90명  

---

### 2) 비율 활용: 각 단계의 비율로 계산

- 5학년 과학 동아리 비율 = 30% × 40% = 0.3 × 0.4 = 0.12 (12%)  
- 6학년 과학 동아리 비율 = 20% × 30% = 0.2 × 0.3 = 0.06 (6%)  
- 전체 과학 동아리 비율 = 12% + 6% = 18%  
- 과학 동아리 학생 수 = 500 × 0.18 = 90명  

---

### 3) 단계별 분해: 작은 단계로 나누어 계산

- 전체 학생 500명  
- 5학년 학생: 500 × 0.3 = 150명  
- 5학년 과학 동아리: 150 × (1 - 0.6) = 150 × 0.4 = 60명  
- 6학년 학생: 500 × 0.2 = 100명  
- 6학년 과학 동아리: 100 × (1 - 0.7) = 100 × 0.3 = 30명  
- 과학 동아리 총 인원 = 60 + 30 = 90명  

---

### 결론:  
세 가지 방법 모두 과학 동아리 학생 수는 **90명**으로 일치합니다.


In [28]:

# [실습 2] gemma2:2b 모델로 Program-Aided Language 구현
pal_template = """
다음 문제를 Python 스타일로 해결하시오:

문제: {question}

def solve_problem():
    # 1. 변수 정의: items = ['farmer', 'wolf', 'sheep', 'cabbage']
    # 2. 이동 시뮬레이션: 각 이동을 리스트로 기록
    # 3. 제약 확인: 안전 조건 체크
    # 4. 총 이동 수 반환

답안:
"""

pal_prompt = PromptTemplate(
    input_variables=["question"],
    template=pal_template
)

pal_chain = pal_prompt | llm
answer = pal_chain.invoke({"question": question})
print("PAL 답안:", answer.content)


PAL 답안: ```python
def solve_problem():
    total_students = 500
    grade_5_ratio = 0.30
    grade_6_ratio = 0.20

    grade_5_students = total_students * grade_5_ratio
    grade_6_students = total_students * grade_6_ratio

    # 5학년 수학 동아리 비율 60%
    grade_5_math = grade_5_students * 0.60
    grade_5_science = grade_5_students - grade_5_math

    # 6학년 수학 동아리 비율 70%
    grade_6_math = grade_6_students * 0.70
    grade_6_science = grade_6_students - grade_6_math

    total_science_club = grade_5_science + grade_6_science

    # 정수로 반환 (학생 수는 정수)
    return int(total_science_club)

# 결과 출력
print(solve_problem())
```

---

**설명:**  
- 전체 학생 500명 중 5학년은 30%, 6학년은 20%로 계산  
- 각 학년별 동아리 비율에 따라 수학/과학 동아리 학생 수 계산  
- 과학 동아리 학생 수를 합산하여 반환  

출력 결과는 과학 동아리 학생 수입니다.


In [29]:

# [실습 2] gemma2:2b 모델로 Reflexion 구현
reflexion_template = """
다음 문제에 대해 초기 답안 작성, 자체 평가, 개선하시오:

문제: {question}

1단계: 초기 답안 - 이동 순서 나열.
2단계: 자체 평가 - 정확성, 논리 오류 확인.
3단계: 개선된 답안 - 평가 기반 수정.

답안:
"""

reflexion_prompt = PromptTemplate(
    input_variables=["question"],
    template=reflexion_template
)

reflexion_chain = reflexion_prompt | llm
answer = reflexion_chain.invoke({"question": question})
print("Reflexion 답안:", answer.content)


Reflexion 답안: 1단계: 초기 답안 - 이동 순서 나열

1. 전체 학생 수: 500명  
2. 5학년 학생 수 = 500 × 30% = 150명  
3. 6학년 학생 수 = 500 × 20% = 100명  
4. 5학년 중 수학 동아리 학생 수 = 150 × 60% = 90명  
5. 5학년 중 과학 동아리 학생 수 = 150 - 90 = 60명  
6. 6학년 중 수학 동아리 학생 수 = 100 × 70% = 70명  
7. 6학년 중 과학 동아리 학생 수 = 100 - 70 = 30명  
8. 과학 동아리 전체 학생 수 = 5학년 과학 동아리 + 6학년 과학 동아리 = 60 + 30 = 90명  

답: 과학 동아리에는 90명의 학생이 있습니다.

---

2단계: 자체 평가

- 정확성: 모든 계산은 문제 조건에 맞게 정확히 수행됨.  
- 논리 오류: 5학년과 6학년 학생 수를 구하고, 각 학년별 동아리 비율을 적용하여 과학 동아리 인원을 구하는 과정에 오류 없음.  
- 누락 사항: 5학년과 6학년 이외의 학생(50%)은 동아리 소속 여부가 문제에 명시되어 있지 않으므로 무시하는 것이 타당함.  
- 문제 해석: 문제에서 요구하는 과학 동아리 학생 수는 5학년과 6학년 학생들만 대상으로 계산하는 것이 맞음.

---

3단계: 개선된 답안

학교에 총 500명의 학생이 있습니다. 이 중 5학년 학생은 30%인 150명, 6학년 학생은 20%인 100명입니다.  
5학년 학생 중 60%는 수학 동아리에, 나머지 40%는 과학 동아리에 속해 있으므로, 5학년 과학 동아리 학생 수는 150 × 40% = 60명입니다.  
6학년 학생 중 70%는 수학 동아리에, 나머지 30%는 과학 동아리에 속해 있으므로, 6학년 과학 동아리 학생 수는 100 × 30% = 30명입니다.  
따라서 과학 동아리 전체 학생 수는 60명 + 30명 = 90명입니다.

**답:** 과학 동아리에는 총 90명의 학생이 있습니다.
